# Analyse de la température maximale annuelle en France : Évolution et tendances

L'analyse présentée dans ce notebook se concentre sur l'évolution des températures maximales annuelles en France, en examinant les données quotidiennes de température relevées dans diverses stations météorologiques entre 1950 et 2023. Les données utilisées proviennent du fichier `Q_64_previous-1950-2023_RR-T-Vent.csv.gz`, disponible à l'adresse https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_64_previous-1950-2023_RR-T-Vent.csv.gz. Ce fichier contient des informations météorologiques quotidiennes, notamment les températures minimales, maximales et moyennes, ainsi que les précipitations et la vitesse du vent.

L'objectif principal de cette analyse est de calculer la moyenne des 10 plus hautes températures maximales par an et de tracer une courbe représentant cette moyenne sur la période étudiée. Cela permettra d'identifier les tendances et les variations dans les températures maximales annuelles en France. L'analyse inclut également un lissage sur 5 ans pour atténuer les fluctuations annuelles et une tendance linéaire pour visualiser l'évolution globale.

## Méthodologie

La méthodologie utilisée dans cette analyse implique plusieurs étapes clés. Tout d'abord, les données sont chargées et filtrées pour ne conserver que les enregistrements avec des valeurs de température maximale (`TX`) non nulles. Ensuite, une requête SQL utilisant DuckDB est exécutée pour calculer la moyenne des 10 plus hautes températures maximales par an. Cette requête implique une sélection des données quotidiennes avec des températures maximales non nulles, un classement de ces températures par ordre décroissant pour chaque année, et enfin, un calcul de la moyenne des 10 premières valeurs pour chaque année. Les résultats sont ensuite visualisés à l'aide de Plotly pour créer une courbe représentant la moyenne annuelle des 10 températures maximales, ainsi qu'une courbe de lissage sur 5 ans et une tendance linéaire.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [2]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_64_previous-1950-2023_RR-T-Vent.csv.gz", safe_mode=True)
print("✅ Données chargées avec succès")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Données chargées avec succès


## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [3]:
# Exécution de la requête
df = run_query(""" WITH daily_max_temp AS (
  SELECT 
    CAST(SUBSTRING("AAAAMMJJ", 1, 4) AS INTEGER) AS year,
    CAST("TX" AS DOUBLE) AS tx_double
  FROM 
    loaded_dataset
  WHERE 
    "TX" IS NOT NULL AND "TX" != ''
),
top_10_temp_per_year AS (
  SELECT 
    year, 
    tx_double,
    ROW_NUMBER() OVER (PARTITION BY year ORDER BY tx_double DESC) AS row_num
  FROM 
    daily_max_temp
)
SELECT 
  year, 
  AVG(tx_double) AS avg_top_10_tx
FROM 
  top_10_temp_per_year
WHERE 
  row_num <= 10
GROUP BY 
  year
ORDER BY 
  year """)
print(f"Résultats : {len(df)} lignes")
df.head()

Résultats : 74 lignes


,year,avg_top_10_tx
0,1950,37.66
1,1951,35.99
2,1952,35.37
3,1953,35.90
4,1954,34.39


## 📈 Visualisation

La bibliothèque principale utilisée est Plotly, qui est adaptée pour créer des visualisations interactives de données. Plotly permet de représenter les données de manière claire et détaillée, avec des fonctionnalités telles que le survol des données pour afficher des informations supplémentaires. Cela rend la visualisation plus engageante et plus facile à comprendre.

In [4]:
import pandas as pd
import duckdb as ddb
import plotly.graph_objects as go
import pandas as pd
import numpy as np

df = df.sort_values('year')

df['moyenne_mobile_5ans'] = df['avg_top_10_tx'].rolling(window=5, center=True).mean()

z = np.polyfit(df['year'], df['avg_top_10_tx'], 1)
p = np.poly1d(z)
df['tendance'] = p(df['year'])

dataviz = go.Figure()

dataviz.add_trace(go.Scatter(
    x=df['year'],
    y=df['avg_top_10_tx'],
    mode='lines+markers',
    name='Moyenne des 10 plus hautes Tx',
    line=dict(color='#d62728', width=2),
    hovertemplate='%{y:.2f} °C<br>année : %{x}<br>'
))

dataviz.add_trace(go.Scatter(
    x=df['year'],
    y=df['moyenne_mobile_5ans'],
    mode='lines',
    name='Lissage 5 ans',
    line=dict(color='steelblue', width=3),
    hovertemplate='%{y:.2f} °C<br>année : %{x}<br>'
))

dataviz.add_trace(go.Scatter(
    x=df['year'],
    y=df['tendance'],
    mode='lines',
    name='Tendance linéaire',
    line=dict(color='forestgreen', width=2, dash='dash'),
    hovertemplate='%{y:.2f} °C<br>année : %{x}<br>'
))

dataviz.update_layout(
    title=dict(
        text="Moyenne annuelle des 10 températures maximales<br>avec lissage sur 5 ans et tendance",
        font=dict(size=16)
    ),
    xaxis_title='Année',
    yaxis_title='Température (°C)',
    hovermode='x unified',
    template='plotly_white',
    margin=dict(l=40, r=40, t=80, b=40),
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=-0.25,
        xanchor='center',
        x=0.5
    )
)
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*

In [5]:

# --- Variables injectées par le script ---
FINAL_OBJECT_VARIABLE_NAME = 'dataviz'
OUTPUT_IMAGE_NAME = 'published\\notebooks\\duckit_analysis_test.png'
OUTPUT_HTML_NAME = 'published\\notebooks\\duckit_analysis_test.html'

# ===================================================================
# CELLULE INJECTÉE AUTOMATIQUEMENT (VERSION ROBUSTE)
# ===================================================================
import sys
import os
# On importe les modules nécessaires pour l'export au cas où
try:
    from bokeh.io import save as bokeh_save
except ImportError:
    bokeh_save = None

try:
    # On s'assure que le dossier de sortie existe
    output_dir = os.path.dirname(OUTPUT_IMAGE_NAME)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    # On utilise globals().get() pour une récupération plus sûre
    final_object = globals().get(FINAL_OBJECT_VARIABLE_NAME)

    if final_object is None:
        # On lève une NameError pour être cohérent avec le code original
        raise NameError(f"name '{FINAL_OBJECT_VARIABLE_NAME}' is not defined")

    print(f"INFO: Variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée. Tentative d'exportation...")

    object_type = str(type(final_object))

    if 'plotly.graph_objs._figure.Figure' in object_type:
        print(f"--> Détecté : Plotly. Sauvegarde HTML et PNG.")
        # 1. Sauvegarde HTML pour l'interactivité
        print(f"--> Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.write_html(OUTPUT_HTML_NAME, include_plotlyjs='cdn')
        # 2. Sauvegarde PNG pour l'aperçu statique
        try:
            print(f"--> Tentative de sauvegarde PNG directe dans : {OUTPUT_IMAGE_NAME}")
            final_object.write_image(OUTPUT_IMAGE_NAME, scale=3, width=1200, height=800)
            print(f"--> Image Plotly sauvegardée avec succès.")
        except Exception as e:
            print(f"AVERTISSESEMENT: La sauvegarde directe en PNG a échoué (kaleido est-il installé?). L'image statique ne sera pas générée.", file=sys.stderr)
            print(f"   Erreur: {e}", file=sys.stderr)
    elif 'folium.folium.Map' in object_type:
        print(f"--> Détecté : Folium. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur pour que le script de post-traitement sache qu'il s'agit de Folium
        with open(f"{OUTPUT_HTML_NAME}.is_folium", "w") as f:
            f.write("true")
    elif 'altair.vegalite' in object_type and hasattr(final_object, 'save'):
        print(f"--> Détecté : Altair. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
    elif 'bokeh.plotting' in object_type and bokeh_save is not None:
        print(f"--> Détecté : Bokeh. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        bokeh_save(final_object, filename=OUTPUT_HTML_NAME, title="")
    elif 'matplotlib.figure.Figure' in object_type:
        print(f"--> Détecté : Matplotlib. Sauvegarde dans : {OUTPUT_IMAGE_NAME}")
        final_object.savefig(OUTPUT_IMAGE_NAME, dpi=300, bbox_inches='tight')
    else:
        print(f"AVERTISSEMENT: Type non supporté : {object_type}", file=sys.stderr)
except NameError:
    print(f"AVERTISSEMENT: Aucune variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée.", file=sys.stderr)
except Exception as e:
    print(f"ERREUR lors de l'exportation : {e}", file=sys.stderr)


INFO: Variable 'dataviz' trouvée. Tentative d'exportation...
--> Détecté : Plotly. Sauvegarde HTML et PNG.
--> Sauvegarde HTML dans : published\notebooks\duckit_analysis_test.html
--> Tentative de sauvegarde PNG directe dans : published\notebooks\duckit_analysis_test.png


--> Image Plotly sauvegardée avec succès.
